In [ ]:
# UNZIP FILES IN TRAIN AND LABEL FOLDERS 
# COPY SPECIFIC FILES TO TRAIN AND LABEL FOLDERS
# FILES: .obj, .json 
# RAW DATA FOLDER: test_raw
# UNZIPPED DATA FOLDER: test_uzip
# SUBFOLDERS: train, label

import os
import shutil
import zipfile
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)
# log.setLevel(logging.WARNING)


log_print = False
root_path = os.path.abspath(os.getcwd())
zip_path = os.path.join(root_path, "test_raw")
data_path = os.path.join(root_path, "test_uzip")
# train_data_path = os.path.join(data_path, "train")
tmp_path = os.path.join(zip_path, "tmp")
# os.makedirs(train_data_path, exist_ok=True)

# Check if there are any folders in test_raw
if not os.listdir(zip_path):
    log.info("No folders found in %s", zip_path)
else:
    for folder in os.listdir(zip_path):
        if 'train' in folder:
            current_train_folder = os.path.join(zip_path, folder)
            log.info(f"current_train_folder: {current_train_folder}")
            log.info(f"number of zip files: {len(os.listdir(current_train_folder))}")
            
            # Check if there are any files in the current_train_folder
            if not os.listdir(current_train_folder):
                log.info(f"No files found in {current_train_folder}")
            else:
                for file in os.listdir(current_train_folder):
                    if 'zip' in file:
                        zip_tmp = os.path.join(current_train_folder, file)
                        train_data_path = os.path.join(data_path, folder)
                        os.makedirs(train_data_path, exist_ok=True)
                        if log_print: log.info(f"file to extract: {zip_tmp}")
                        if log_print: log.info(f"file to folder: {train_data_path}")
                        
                        try:
                            with zipfile.ZipFile(zip_tmp, 'r') as zip_ref:
                                zip_ref.extractall(tmp_path)
                                for file in os.listdir(tmp_path):
                                    if 'obj' in file:
                                        file_tocp = os.path.join(tmp_path, file)
                                        shutil.copy(file_tocp, train_data_path)
                                    os.remove(tmp_path + '/' + file)
                            if log_print: log.info(f"Successfully unzipped {file_tocp} to {train_data_path}")
                        except Exception as e:
                            log.error(f"Error unzipping {file_tocp}: {e}")
                log.info(f"number of copied {folder} files: {len(os.listdir(train_data_path))}")
                
        elif 'label' in folder: 
            current_label_folder = os.path.join(zip_path, folder)
            log.info(f"current_label_folder: {current_label_folder}")
            log.info(f"number of zip files: {len(os.listdir(current_label_folder))}")
            if not os.listdir(current_label_folder):
                log.info(f"No files found in {current_label_folder}")
            else:
                for file in os.listdir(current_label_folder):
                    if 'zip' in file and '_2D' not in file and 'META' not in file:
                        zip_tmp = os.path.join(current_label_folder, file)
                        label_data_path= os.path.join(data_path, folder)
                        os.makedirs(label_data_path, exist_ok=True)
                        if log_print: log.info(f"file to extract: {zip_tmp}")
                        if log_print: log.info(f"file to folder: {label_data_path}")
                        
                        try:
                            with zipfile.ZipFile(zip_tmp, 'r') as zip_ref:
                                zip_ref.extractall(tmp_path)
                                for file in os.listdir(tmp_path):
                                    if 'mpie68.json' in file:
                                        file_tocp = os.path.join(tmp_path, file)
                                        shutil.copy(file_tocp, label_data_path)
                                    os.remove(tmp_path + '/' + file)
                            if log_print: log.info(f"Successfully unzipped {file_tocp} to {label_data_path}")
                        except Exception as e:
                            log.error(f"Error unzipping {file_tocp}: {e}")
                log.info(f"number of copied {folder} files: {len(os.listdir(label_data_path))}")
            


In [ ]:

# EXTRA FUNCTIONS FOR DATA LOCATION PREPARATION
def print_max_lnd(data):
    landmarks = data["landmarks"]
    
    x_values = [point["x"] for point in landmarks]
    y_values = [point["y"] for point in landmarks]
    z_values = [point["z"] for point in landmarks]

    x_range = (min(x_values), max(x_values))
    y_range = (min(y_values), max(y_values))
    z_range = (min(z_values), max(z_values))

    print(f"X range: {x_range}")
    print(f"Y range: {y_range}")
    print(f"Z range: {z_range}")
    
def normalize_points(points):
    """Normalize a list of points (x, y, z) to have a mean of 0 and a standard deviation of 1"""
    points_np = np.array(points)
    mean = np.mean(points_np, axis=0)
    std_dev = np.std(points_np, axis=0)
    normalized_points = (points_np - mean) / std_dev
    return normalized_points.tolist()

In [1]:
# REED COCO FORMAT 3D LANDMARKS FROM JSON FILE IN LABEL FOLDER
# CONVERT LANDMARKS TO ARRAY
# SAVE 3D ARRAY TO NPY FILE
# FILES:  .json, .npy
# RAW DATA FOLDER: test_unzip/label
# CONVERTED DATA FOLDER: test_unzip/label (the same folder)

import os
import json
import numpy as np
import logging

root_path = os.path.abspath(os.getcwd())
data_path = os.path.join(root_path, "test_uzip")
json_path = os.path.join(data_path, "label")
log_print = False

# Set up logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)
# log.setLevel(logging.WARNING)

# Check if there are any folders in test_raw
if not os.listdir(json_path):
    log.info("No folders found in %s", json_path)
else:
    for json_file in os.listdir(json_path):
        if 'mpie68.json' in json_file:
            json_filepath = os.path.join(json_path, json_file)
            
            # Open and read the JSON file
            with open(json_filepath, 'r') as file:
                data = json.load(file)
                
                # show max and min values of landmarks
                # print_max_lnd(data)
                
                # Extract landmarks
                landmarks_data = data.get('landmarks', [])
                
                # Convert landmarks to a list of [x, y, z] and then to a numpy array
                landmarks_list = [[point['x'], point['y'], point['z']] for point in landmarks_data]
                landmarks_array = np.array(landmarks_list)
                
                # normalize landmarks
                # landmarks_array = normalize_points(landmarks_array)
                
                # Save the numpy array to an .npy file
                npy_filepath = os.path.join(json_path, json_file.replace('.json', '.npy'))
                np.save(npy_filepath, landmarks_array)
                
                if log_print:
                    log.info(f"Saved landmarks from {json_file} to {npy_filepath}")
    log.info(f"number of converted {json_file} files: {len(os.listdir(json_path))}")



INFO:__main__:number of converted 1107M_FC_O-mpie68.json files: 54


In [4]:
import os

root = os.path.abspath(os.getcwd())
obj_new = os.path.join(root, "example/out_faceE.obj")
print('\n\n|------------- obj_n:', obj_new)

obj_in = os.path.join(root, "test_uzip/train/1107M_FC_A.obj")

# Read the file
with open(obj_in, 'r') as file:
    lines = file.readlines()

inside_mouth_socket = False
deleted_vertex_indices = set()
vertex_count = 0
filtered_lines = []

for line in lines:
    if "g mouth_socket" in line:
        inside_mouth_socket = True
        continue
    if "g " in line and "mouth_socket" not in line:
        inside_mouth_socket = False
    if line.startswith("v "):
        vertex_count += 1
        if inside_mouth_socket:
            deleted_vertex_indices.add(vertex_count)
            continue
    if not inside_mouth_socket:
        filtered_lines.append(line)

# Adjust face indices
adjusted_lines = []
for line in filtered_lines:
    if line.startswith("f "):
        parts = line.split()
        face_vertices = [int(part.split('/')[0]) for part in parts[1:]]
        if any(idx in deleted_vertex_indices for idx in face_vertices):
            continue  # Skip faces that reference deleted vertices
        adjusted_parts = ["f"]
        for part in parts[1:]:
            vertex_index = int(part.split('/')[0])
            adjusted_vertex_index = vertex_index - len([idx for idx in deleted_vertex_indices if idx < vertex_index])
            adjusted_parts.append(part.replace(str(vertex_index), str(adjusted_vertex_index)))
        adjusted_lines.append(' '.join(adjusted_parts) + '\n')
    else:
        adjusted_lines.append(line)

# Write the adjusted lines back to the file
with open(obj_new, 'w') as file:
    file.writelines(adjusted_lines)




|------------- obj_n: /home/jakhon37/myprojects/3DFACE/mica_DEV/MICA/dataset/example/out_faceE.obj


In [40]:
# REED 2 OBJ FACE MESHES 
# 1ST OBJ FILE IS UNNORMALIZED
# 2ND OBJ FILE IS NORMALIZED
# NEED TO NORMALIZE 1ST OBJ FILE BAED ON 2ND OBJ FILE

import util
import os 
import numpy as np
import logging
import shutil

def load_obj(obj_path):
    vertices, textures, faces = [], [], []
    with open(obj_path, 'r') as fp:
        for line in fp:
            line_split = line.split()
            if not line_split:
                continue
            elif line_split[0] == 'v':
                vertices.append(list(map(float, line_split[1:4])))
            elif line_split[0] == 'vt':
                textures.append(list(map(float, line_split[1:3])))
            elif line_split[0] == 'f':
                face = [list(map(int, vert.split('/'))) for vert in line_split[1:]]
                faces.append(face)
    # return np.array(vertices), np.array(textures), faces
    vertices, textures, faces = np.array(vertices), np.array(textures), faces
    print('\nlen of vertices:', len(vertices))
    print('len of faces:', len(faces))
    print('len of textures:', len(textures))
    return vertices, textures, faces



def normalize_obj_and_landmarks(obj_vert, obj_n_vert, landmarks):
    # Convert to numpy arrays
    vertices1_np = np.array(obj_vert)
    vertices2_np = np.array(obj_n_vert)
    landmarks_np = np.array(landmarks)

    # Calculate mean and range for the unnormalized vertices
    mean1 = np.mean(vertices1_np, axis=0)
    range1 = np.max(vertices1_np, axis=0) - np.min(vertices1_np, axis=0)

    # Normalize the unnormalized vertices
    normalized_vertices1 = (vertices1_np - mean1) / range1

    # Normalize the landmarks using the same mean and range
    normalized_landmarks = (landmarks_np - mean1) / range1

    # Calculate mean and range for the normalized vertices
    mean2 = np.mean(vertices2_np, axis=0)
    range2 = np.max(vertices2_np, axis=0) - np.min(vertices2_np, axis=0)
    print('\n\n|------------- mean2:', mean2)
    print('|------------- range2:', range2)

    # Scale and translate the normalized vertices to match the second set
    transformed_vertices1 = normalized_vertices1 * range2 + mean2

    # Scale and translate the normalized landmarks to match the second set
    transformed_landmarks = normalized_landmarks * range2 + mean2

    return transformed_vertices1.tolist(), transformed_landmarks.tolist()


# 0-16 - jaw points 
# 17-26  - eye brow points 
# 27-35 - nose points 
# 36-47 -  eye points 
# 48-67 -  mouth points 

# for 68 face points mapping 


# eye brow: 1-10
# nose: 11-19
# eye: 20-31
# outer mouth: 32-43
# inner mouth: 44-51 

# for flame 51 face points mapping

def map_68_to_51(landmarks_68):
    # A potential mapping from 68-point to 51-point landmarks
    # This is a general mapping and might need adjustments based on the specific FLAME configuration
    flame_mapping = [
        # Eyebrow (5 points from each eyebrow)
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
        # Nose
        27, 28, 29, 30, 31, 32, 33, 34, 35,
        # Eyes (6 points from each eye)
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        # Outer lip
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
        # Inner lip
        60, 61, 62, 63, 64, 63, 62, 61
        # 60, 61, 62, 63, 64, 65, 66, 67
    ]

    # Extract the 51-point landmarks from the 68-point set
    landmarks_51 = [landmarks_68[i] for i in flame_mapping]
    return landmarks_51
# eye brow: 1-10
# nose: 11-19
# eye: 20-31
# outer mouth: 32-43
# inner mouth: 44-51        


def write_obj(filename, vertices, faces=None):
    """
    Write vertices and faces to an OBJ file.

    :param filename: Name of the OBJ file to write to.
    :param vertices: List of vertices. Each vertex is a tuple of (x, y, z) coordinates.
    :param faces: List of faces. Each face is a tuple of vertex indices. (Optional)
    """
    with open(filename, 'w') as f:
        # Write vertices to the file
        for vertex in vertices:
            f.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

        # If faces are provided, write them to the file
        if faces:
            # for face in faces:
            #     f.write(f"f {' '.join(map(str, face))}\n")

                # Write faces to the file
            for face in faces:
                face_str = ' '.join([f"{idx[0]}/{idx[1]}" for idx in face])
                f.write(f"f {face_str}\n")
            
root = os.path.abspath(os.getcwd())
obj_n = os.path.join(root, "example/hello_flame.obj")
print('\n\n|------------- obj_n:', obj_n)

obj_ = os.path.join(root, "test_uzip/train/1107M_FC_A.obj")
obj_ = os.path.join(root, "test_uzip/train/1125M_FC_A.obj")
obj_ = os.path.join(root, "example/1107M_FC_A.obj")
obj_ = os.path.join(root, "example/out_face.obj")

lnd3d = os.path.join(root, "test_uzip/label/1107M_FC_A-mpie68.npy")
# lnd3d = os.path.join(root, "test_uzip/label/1125M_FC_A-mpie68.npy")
print('\n\n|------------- obj_n:', obj_)

nrm_obj = os.path.join(root, "example/nrm_n07.obj")
nrm3dlnd = os.path.join(root, "example/nrm_n-mpie68_07.npy")


vertices_n, textures_n, faces_n = load_obj(obj_n)
vertices_, textures_, faces_ = load_obj(obj_)
landmarks3d = np.load(lnd3d)
print('\n\n|------------- landmarks3d:', len(landmarks3d))
print('\n\n|------------- vertices_n:', len(vertices_n))
print('|------------- vertices_:', len(vertices_))
print('|------------- faces_n:', len(faces_n))
print('|------------- faces_:', len(faces_))

transformed_vertices_, tformed_3dlnd = normalize_obj_and_landmarks(vertices_, vertices_n, landmarks3d)

# tformed_3dlnd = [[round(val, 7) for val in row] for row in tformed_3dlnd]
transformed_3dlnd_51 = map_68_to_51(tformed_3dlnd)
# for row in transformed_3dlnd:
#     print(f"[ {row[0]: <10} {row[1]: <10} {row[2]: <10} ]")
# print('\n\n|------------- transformed_vertices_:', transformed_3dlnd_51)
    
# transformed_vertices_ = normlize_obj(vertices_n, vertices_)
# transformed_vertices_ = transformed_vertices_.tolist()
print('len of transformed_vertices_:', len(transformed_vertices_))
print('len of faces_:', len(faces_))
print('len of transformed_3dlnd:', len(transformed_3dlnd_51))

write_obj(nrm_obj, transformed_vertices_, faces_)
np.save(nrm3dlnd, transformed_3dlnd_51) 





|------------- obj_n: /home/jakhon37/myprojects/3DFACE/mica_DEV/MICA/dataset/example/hello_flame.obj


|------------- obj_n: /home/jakhon37/myprojects/3DFACE/mica_DEV/MICA/dataset/example/out_face.obj

len of vertices: 5023
len of faces: 9976
len of textures: 0

len of vertices: 19275
len of faces: 16858
len of textures: 19500


|------------- landmarks3d: 68


|------------- vertices_n: 5023
|------------- vertices_: 19275
|------------- faces_n: 9976
|------------- faces_: 16858


|------------- mean2: [-0.00563593 -0.00093774  0.00206536]
|------------- range2: [0.229568 0.332232 0.236763]
len of transformed_vertices_: 19275
len of faces_: 16858
len of transformed_3dlnd: 51


In [38]:
# ADJUST FACES AND VERTICES IN OBJ FILE
# RESCALE TO FLAME SCALE
# FILES: .obj

# ! pip install open3d
import os 
import trimesh
import numpy as np


def add_face_to_mesh(mesh):
    # Get the last face
    last_face = mesh.faces[-1]
    
    # Duplicate the last face
    new_face = last_face.copy()
    
    # Add the new face to the mesh
    mesh.faces = np.vstack([mesh.faces, new_face])
    
    return mesh


def rescale_tofalme(inObjPath, outObjPath, ad_vert=False,  add_face=False):
    # Load the original mesh
    mesh = trimesh.load_mesh(inObjPath)
    
    # Decimate the mesh
    decimated_mesh = mesh.simplify_quadratic_decimation(face_count=9950)  # target number of faces 9552
    
    # Add an extra face if desired faces has not been reached
    if add_face:
        decimated_mesh = add_face_to_mesh(decimated_mesh)
    
    # Save the decimated mesh
    decimated_mesh.export(outObjPath)
    
    if ad_vert:
        # Duplicate the last vertex
        new_vertex = decimated_mesh.vertices[-1]
        # Add the new vertex to the vertices array
        decimated_mesh.vertices = np.vstack([decimated_mesh.vertices, new_vertex])
        # Save the modified mesh
        decimated_mesh.export(outObjPath)


root = os.path.abspath(os.getcwd())
obj_in = os.path.join(root, "example/nrm_n07.obj")
# obj_in = os.path.join(root, "example/fitting_exmpl.obj")
obj_out = os.path.join(root, "example/nrm_n07_scaled.obj")
# obj_out = os.path.join(root, "example/fitting_ex.obj")

rescale_tofalme(obj_in, obj_out,  ad_vert=False, add_face=False)
print('\n\n|------------- obj_out:', obj_out)



|------------- obj_out: /home/jakhon37/myprojects/3DFACE/mica_DEV/MICA/dataset/example/nrm_n07_scaled.obj


In [ ]:
# CONVERT OBJ TO PLY USING PYVISTA
# FILES: .obj, .ply

import pyvista as pv
import pyvista as pv
import os
import numpy as np
import trimesh


root = os.path.abspath(os.getcwd())
obj_in = os.path.join(root, "example/nrm_n07_scaled.obj")
obj_in = os.path.join(root, "example/nrm_n07.obj")
# obj_in = os.path.join(root, "example/nrm_n07.obj")
obj_out = os.path.join(root, "example/nrm_n07_scaled.ply")
# rescale_tofalme2(obj_in, obj_out, ad_vert=True)
print('\n\n|------------- obj_out:', obj_out)
# Load the original mesh
mesh = pv.read(obj_in)

# Decimate the mesh
decimated_mesh = mesh.decimate(target_reduction=1.0 - (9976 / mesh.n_faces))

# Save the decimated mesh
decimated_mesh.save(obj_out)



|------------- obj_out: /home/jakhon37/myprojects/3DFACE/mica_DEV/MICA/dataset/example/nrm_n07_scaled.ply


: 

: 

In [1]:
# CONVERT 3D OBJ TO 3D PLY FORMAT
# FILES: .obj, .ply

import os
import numpy as np
import logging

def write_ply(filename, vertices, faces=None):
    """
    Write vertices and faces to a PLY file.

    :param filename: Name of the PLY file to write to.
    :param vertices: List of vertices. Each vertex is a tuple of (x, y, z) coordinates.
    :param faces: List of faces. Each face is a tuple of vertex indices. (Optional)
    """
    with open(filename, 'w') as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(vertices)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        if faces:
            f.write(f"element face {len(faces)}\n")
            f.write("property list uchar int vertex_index\n")
        f.write("end_header\n")

        # Write vertices to the file
        for vertex in vertices:
            f.write(f"{vertex[0]} {vertex[1]} {vertex[2]}\n")

        # If faces are provided, write them to the file
        if faces:
            for face in faces:
                face_str = ' '.join(map(str, [idx[0] for idx in face]))
                f.write(f"{len(face)} {face_str}\n")

def load_obj(obj_path):
    vertices, textures, faces = [], [], []
    with open(obj_path, 'r') as fp:
        for line in fp:
            line_split = line.split()
            if not line_split:
                continue
            elif line_split[0] == 'v':
                vertices.append(list(map(float, line_split[1:4])))
            elif line_split[0] == 'vt':
                textures.append(list(map(float, line_split[1:3])))
            elif line_split[0] == 'f':
                # Adjusting the vertex indices by subtracting 1
                face = [list(map(lambda x: int(x)-1 if x.isdigit() else x, vert.split('/'))) for vert in line_split[1:]]
                faces.append(face)
    vertices, textures, faces = np.array(vertices), np.array(textures), faces
    print('\nlen of vertices:', len(vertices))
    print('len of faces:', len(faces))
    print('len of textures:', len(textures))
    return vertices, textures, faces


root = os.path.abspath(os.getcwd())

obj_path = os.path.join(root, "example/nrm_n07.obj")
objOut = os.path.join(root, "example/nrm_n07_scaled.ply")
print('\n\n|------------- obj_path:', obj_path)
# Load the OBJ file
vertices, _, faces = load_obj(obj_path)

# Convert to PLY format and save
write_ply(objOut, vertices, faces)




|------------- obj_path: /home/jakhon37/myprojects/3DFACE/mica_DEV/MICA/dataset/example/nrm_n07.obj

len of vertices: 19275
len of faces: 16858
len of textures: 0


In [ ]:
# ! pip install plotly
# VISUALIZE 3D LANDMARKS
# FILES: .npy

import plotly.graph_objects as go
import numpy as np
import os

def plot_lnd_points_interactive(lndmark):
    trace = go.Scatter3d(
        x=lndmark[:, 0],
        y=lndmark[:, 1],
        z=lndmark[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.8
        )
    )

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=[trace], layout=layout)
    fig.show()

root = os.path.abspath(os.getcwd())
lnd3d = os.path.join(root, "test_uzip/label/1107M_FC_A-mpie68.npy")
lnd3d = os.path.join(root, "example/nrm_n-mpie68.npy")


landmarks3d = np.load(lnd3d)
plot_lnd_points_interactive(landmarks3d)
